In [1]:
#| default_exp interface_pymeshlab

In [2]:
#| export

from blender_tissue_cartography import io as tcio
import numpy as np
import pymeshlab
import warnings

Unable to load the following plugins:

	libio_e57.so: libio_e57.so does not seem to be a Qt Plugin.

Cannot load library /home/nikolas/Programs/miniconda3/envs/blender-tissue-cartography/lib/python3.11/site-packages/pymeshlab/lib/plugins/libio_e57.so: (/lib/x86_64-linux-gnu/libp11-kit.so.0: undefined symbol: ffi_type_pointer, version LIBFFI_BASE_7.0)



In [3]:
from importlib import reload

In [4]:
np.set_printoptions(suppress=True)

## Interfacing with `pymeshlab`

In this notebook, we define functions to convert our `ObjMesh` class to and from `pymeshlab`.

Unfortunately, `pymeshlab`'s default IO discards UV information. So just relying on writing stuff to disk won't work.

In [5]:
reload(tcio)

<module 'blender_tissue_cartography.io' from '/home/nikolas/Documents/UCSB/streichan/numerics/code/python code/jupyter notebooks/blender-tissue-cartography/blender_tissue_cartography/io.py'>

In [6]:
mesh_fname_data = "registration_example/Drosophila_CAAX-mCherry_mesh_remeshed.obj"
mesh_fname_ref = "registration_example/Drosophila_reference_preregistered.obj"

In [7]:
mesh_data = tcio.ObjMesh.read_obj(mesh_fname_data)
mesh_ref = tcio.ObjMesh.read_obj(mesh_fname_ref)

In [36]:
#| export

def convert_to_pymeshlab(mesh: tcio.ObjMesh, add_texture_info=None) -> pymeshlab.Mesh:
    """
    Convert tcio.ObjMesh to pymeshlab.Mesh.
    
    See https://pymeshlab.readthedocs.io/en/latest/classes/mesh.html
    Note: normal information is recalculated by pymeshlab. Discards any non-triangle faces.
    
    Texture is saved as a vertex attribute via v_tex_coords_matrix. Note that this discards
    information since a vertex can have multiple texture coordinates. For this reason,
    we also save it as 3 custom face attributes 'face_tex_vertex_{0/1/2}'.
    
    Parameters
    ----------
    mesh : tcio.ObjMesh
    add_texture_info : None or bool
        Whether to add texture info to the pymeshlab.Mesh. If None, texture is added if
        available for at least one vertex.
    Returns
    -------
    pymeshlab.Mesh

    """
    if not mesh.is_triangular:
        warnings.warn(f"Warning: mesh not triangular. discarding non-triangular faces")
    add_texture_info = ((not mesh.only_vertices and len(mesh.texture_vertices) > 0)
                        if add_texture_info is None else add_texture_info)
    if not add_texture_info:
        return pymeshlab.Mesh(vertex_matrix=mesh.vertices, face_matrix=mesh.tris)
    converted = pymeshlab.Mesh(vertex_matrix=mesh.vertices, face_matrix=mesh.tris,
                               v_tex_coords_matrix=mesh.vertex_textures)
    texture_tris = mesh.texture_tris
    for vertex in [0,1,2]:
        atttrib = np.pad(tcio.index_else_nan(mesh.texture_vertices, texture_tris[:,vertex]),
                         ((0,0), (0,1)), constant_values=0)
        converted.add_face_custom_point_attribute(atttrib, f"face_tex_vertex_{vertex}")
    return converted

In [16]:
pymesh_data = convert_to_pymeshlab(mesh_data)

In [37]:
%%time
pymesh_ref = convert_to_pymeshlab(mesh_ref)

CPU times: user 356 ms, sys: 3.97 ms, total: 360 ms
Wall time: 360 ms


In [97]:
np.allclose(mesh_ref.vertices, pymesh_ref.vertex_matrix())

True

In [98]:
pymesh_normals = pymesh_ref.vertex_normal_matrix()
pymesh_normals = (pymesh_normals.T / np.linalg.norm(pymesh_normals, axis=-1)).T

np.einsum('vi,vi->v', mesh_ref.vertex_normals, pymesh_normals)

array([0.99904184, 0.99822658, 0.99668032, ..., 0.99417082, 0.99599803,
       0.99721726])

In [99]:
np.allclose(mesh_ref.vertex_textures, pymesh_ref.vertex_tex_coord_matrix())

True

In [81]:
%%timeit
convert_to_pymeshlab(mesh_ref)

746 ms ± 3.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [44]:
#| export

def convert_from_pymeshlab(mesh: pymeshlab.Mesh, reconstruct_texture_from_faces=True,
                           texture_vertex_decimals=10) -> tcio.ObjMesh:
    """
    Convert pymeshlab mesh to ObjMesh.
    
    Texture vertices can be reconstructed from face attribute face_tex_vertex_{0/1/2} or from
    vertex attribute vertex_tex_coord_matrix. Reconstruction from face texture can accomodate
    multiple texture coordinates per vertex (e.g. for UV maps with seams).
    
    Texture vertices are rounded to texture_vertex_decimals decimals
    """
    vertices = mesh.vertex_matrix()
    normals = mesh.vertex_normal_matrix()
    normals = (normals.T / np.linalg.norm(normals, axis=-1)).T
    if not mesh.has_vertex_tex_coord():
        faces = [[3*[v,] for v in f] for f in mesh.face_matrix()]
        return tcio.ObjMesh(vertices=vertices, faces=faces, normals=normals)
    if mesh.has_vertex_tex_coord() and not reconstruct_texture_from_faces:
        faces = [[3*[v,] for v in f] for f in mesh.face_matrix()]
        return tcio.ObjMesh(vertices=vertices, faces=faces, normals=normals,
                           texture_vertices=mesh.vertex_tex_coord_matrix())
    # reconstruct texture vertices - big pain.
    texture_vertices = np.vstack([mesh.face_custom_point_attribute_matrix(f"face_tex_vertex_{i}")[:,:2]
                                  for i in [0,1,2]])
    texture_vertices = np.round(texture_vertices, decimals=texture_vertex_decimals)
    texture_vertices_unique, inverse_index = np.unique(texture_vertices, axis=0, return_inverse=True)
    
    n_faces = mesh.face_matrix().shape[0]
    faces = [[[v, inverse_index[ifc+iv*n_faces], v] for iv, v in enumerate(fc)]
             for ifc, fc in enumerate(mesh.face_matrix())]

    return tcio.ObjMesh(vertices=vertices, faces=faces, normals=normals, texture_vertices=texture_vertices_unique)

In [45]:
pymesh_ref = convert_to_pymeshlab(mesh_ref)

In [47]:
%%time
convert_from_pymeshlab(pymesh_ref)

CPU times: user 571 ms, sys: 20 ms, total: 591 ms
Wall time: 591 ms


In [40]:
mesh_seams = tcio.ObjMesh.read_obj("drosophila_example/Drosophila_CAAX-mCherry_mesh_uv.obj")

In [41]:
pymesh_seams = convert_to_pymeshlab(mesh_seams,add_texture_info=True)

In [42]:
convert_from_pymeshlab(pymesh_seams, reconstruct_texture_from_faces=False).texture_vertices.shape

/tmp/ipykernel_1257322/2081661155.py:16: RuntimeWarning: invalid value encountered in divide
  normals = (normals.T / np.linalg.norm(normals, axis=-1)).T


(8160, 2)

In [43]:
mesh_seams_reconverted = convert_from_pymeshlab(pymesh_seams, reconstruct_texture_from_faces=True)

/tmp/ipykernel_1257322/2081661155.py:16: RuntimeWarning: invalid value encountered in divide
  normals = (normals.T / np.linalg.norm(normals, axis=-1)).T


In [260]:
mesh_seams.texture_vertices.shape, mesh_seams_reconverted.texture_vertices.shape

((8288, 2), (8276, 2))

In [262]:
mesh_seams_reconverted.write_obj("drosophila_example/Drosophila_CAAX-mCherry_mesh_uv_resaved.obj")

In [274]:
pymesh_seams.face_custom_point_attribute_matrix(f"face_tex_vertex_{1}").shape

(16274, 3)

You can now use `MeshLab` filters like this, and the face attributes will hopefully be correctly updated.

In [304]:
ms = pymeshlab.MeshSet()
ms.add_mesh(pymesh_seams)

ms.meshing_merge_close_vertices(threshold=pymeshlab.PercentageValue(100))
pymesh_seams_remeshed = ms.current_mesh()

In [305]:
pymesh_seams_remeshed.face_custom_point_attribute_matrix(f"face_tex_vertex_{1}").shape

(9054, 3)

In [312]:
convert_from_pymeshlab(pymesh_seams_remeshed, reconstruct_texture_from_faces=True,
                       texture_vertex_decimals=12).texture_vertices.shape

/tmp/ipykernel_1145232/2081661155.py:16: RuntimeWarning: invalid value encountered in divide
  normals = (normals.T / np.linalg.norm(normals, axis=-1)).T


(8072, 2)